<a href="https://colab.research.google.com/github/franbusleiman/Araucaria/blob/main/Weather_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.7 MB/s eta 0:00:00


In [14]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below

In [15]:
def request_weather(latitude, longitude):
    url = "https://customer-archive-api.open-meteo.com/v1/archive"
    params = {
	    "latitude": 52.52,
	    "longitude": 13.41,
	    "start_date": "2023-01-01",
	    "end_date": "2023-12-31",
	    "hourly": ["temperature_2m", "relative_humidity_2m", "precipitation"],
	    "apikey": "sGnjvxrP98MufeQJ"
      }
    responses = openmeteo.weather_api(url, params=params)

    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]
    print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
    print(f"Elevation {response.Elevation()} m asl")
    print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
    print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

    # Process hourly data. The order of variables needs to be the same as requested.
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
    hourly_precipation = hourly.Variables(2).ValuesAsNumpy()

    hourly_data = {"date": pd.date_range(
	    start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	    end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	    freq = pd.Timedelta(seconds = hourly.Interval()),
	    inclusive = "left"
    )}
    hourly_data["temperature_2m"] = hourly_temperature_2m
    hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
    hourly_data["precipation"] = hourly_precipation


    hourly_dataframe = pd.DataFrame(data = hourly_data)
    return hourly_dataframe;



In [16]:
latitude= 52.52
longitude= 13.41
data = request_weather(latitude, longitude)
print(data)


Coordinates 52.5483283996582°N 13.407821655273438°E
Elevation 38.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                          date  temperature_2m  relative_humidity_2m  \
0    2023-01-01 00:00:00+00:00         15.3085             58.668945   
1    2023-01-01 01:00:00+00:00         15.7085             54.692139   
2    2023-01-01 02:00:00+00:00         15.7585             52.490711   
3    2023-01-01 03:00:00+00:00         15.8585             50.558018   
4    2023-01-01 04:00:00+00:00         15.8085             49.503017   
...                        ...             ...                   ...   
8755 2023-12-31 19:00:00+00:00          6.3085             83.442131   
8756 2023-12-31 20:00:00+00:00          6.6585             84.960327   
8757 2023-12-31 21:00:00+00:00          6.0585             87.312302   
8758 2023-12-31 22:00:00+00:00          5.2085             88.159035   
8759 2023-12-31 23:00:00+00:00          5.1585             86.918320   

      prec